#### COMP8420 - Advanced Natural Language Processing
#### Session 1, 2024
# Major Project
---

## Revolutionizing IT Support with an AI- Powered Chatbot

#### Leveraging NLP for Efficient Ticket Triage and Resolution
___

#### Group Members:

### Mir Sadia Afrin - 47965495
### Fursan Afzal - 47748893


In [1]:
!pip install transformers

In [2]:
!pip install transformers torch

## Necessary Libraries

In [14]:
import pandas as pd
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch

## Loading and Preprocessing the Dataset

In [26]:
# Load the dataset
df = pd.read_csv('Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv')

# Inspect the dataset
df.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [40]:
# Preprocess the data
def preprocess_text(text):
    # Implement text cleaning steps
    return text

df['instruction'] = df['instruction'].apply(preprocess_text)
df['response'] = df['response'].apply(preprocess_text)

# Tokenization
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

def tokenize_function(examples):
    return tokenizer(examples, padding='max_length', truncation=True, max_length=128)

train_encodings = tokenize_function(train_df['instruction'].tolist())
train_labels = tokenize_function(train_df['response'].tolist())

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = CustomDataset(train_encodings, train_labels)

In [6]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/16125 [00:00<?, ?it/s]

{'loss': 4.7942, 'grad_norm': 23.240041732788086, 'learning_rate': 4.8449612403100775e-05, 'epoch': 0.09}
{'loss': 4.3643, 'grad_norm': 11.681827545166016, 'learning_rate': 4.6899224806201553e-05, 'epoch': 0.19}
{'loss': 4.2739, 'grad_norm': 8.936216354370117, 'learning_rate': 4.5348837209302326e-05, 'epoch': 0.28}
{'loss': 4.1199, 'grad_norm': 18.024036407470703, 'learning_rate': 4.3798449612403104e-05, 'epoch': 0.37}
{'loss': 4.0832, 'grad_norm': 5.541406154632568, 'learning_rate': 4.2248062015503877e-05, 'epoch': 0.47}
{'loss': 4.0011, 'grad_norm': 4.467293739318848, 'learning_rate': 4.0697674418604655e-05, 'epoch': 0.56}
{'loss': 3.9861, 'grad_norm': 19.31856918334961, 'learning_rate': 3.914728682170543e-05, 'epoch': 0.65}
{'loss': 3.9742, 'grad_norm': 6.0319504737854, 'learning_rate': 3.7596899224806207e-05, 'epoch': 0.74}
{'loss': 3.9584, 'grad_norm': 7.928410053253174, 'learning_rate': 3.604651162790698e-05, 'epoch': 0.84}
{'loss': 3.8854, 'grad_norm': 4.654712200164795, 'learni

TrainOutput(global_step=16125, training_loss=3.8980842739371364, metrics={'train_runtime': 59376.9554, 'train_samples_per_second': 1.086, 'train_steps_per_second': 0.272, 'total_flos': 4212746108928000.0, 'train_loss': 3.8980842739371364, 'epoch': 3.0})

## Evaluating the Model on the Test Set

In [46]:
test_encodings = tokenize_function(test_df['instruction'].tolist())
test_labels = tokenize_function(test_df['response'].tolist())

test_dataset = CustomDataset(test_encodings, test_labels)

# Evaluate the model
eval_results = trainer.evaluate(test_dataset)
print(eval_results)

  0%|          | 0/672 [00:00<?, ?it/s]

{'eval_loss': 3.7349560260772705, 'eval_runtime': 566.5196, 'eval_samples_per_second': 9.488, 'eval_steps_per_second': 1.186, 'epoch': 3.0}


## Creating Response Generation

In [47]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Select the appropriate device
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

print(f"Using device: {device}")

Using device: mps


In [48]:
# Load the model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Move the model to the selected device
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [51]:
# Function to generate responses
def generate_response(prompt, model, tokenizer, max_length=150):
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    
    # Move the input IDs and attention mask to the correct device
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Ensure max_length does not exceed model's max position embeddings
    max_length = min(max_length, model.config.n_positions)
    
    # Generate a response using beam search
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length,
        num_beams=5,              # Use beam search with 5 beams
        early_stopping=True,      # Stop when at least num_beams sentences are finished per batch
        no_repeat_ngram_size=2,   # Prevents repeating n-grams
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Decode the generated tokens to text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response

# Test the function
query = "I want a refund for my recent purchase."
response = generate_response(query, model, tokenizer)
print(f"Query: {query}\nResponse: {response}")

Query: I want a refund for my recent purchase.
Response: I want a refund for my recent purchase.

Rated 5 out of 5 by Anonymous from Great product! I bought this product because I was looking for a way to get a better quality product. I have been using it for about a month now and it works great. The only thing I would change is the size of the tip. It is a little smaller than the one on the back of my phone, but it is still a good size for me. If you have any questions, please don't hesitate to call us. We will be happy to help you with any issues you may have.


In [50]:
# Function to generate responses
def generate_response(prompt, model, tokenizer, max_length=150):
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    
    # Move the input IDs and attention mask to the correct device
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Ensure max_length does not exceed model's max position embeddings
    max_length = min(max_length, model.config.n_positions)
    
    # Generate a response using beam search
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length,
        num_beams=5,               # Use beam search with 5 beams
        early_stopping=True,       # Stop when at least num_beams sentences are finished per batch
        no_repeat_ngram_size=2,    # Prevents repeating n-grams
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Decode the generated tokens to text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response

# Load the dataset
df = pd.read_csv('Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv')

# Sample a subset for testing
test_sample = df.sample(10, random_state=42)

# Function to test the response generation
def test_response_generation(test_sample, model, tokenizer):
    results = []
    for idx, row in test_sample.iterrows():
        prompt = row['instruction']  # Use instruction column as prompt
        expected_response = row['response']
        generated_response = generate_response(prompt, model, tokenizer)
        
        results.append({
            'prompt': prompt,
            'expected_response': expected_response,
            'generated_response': generated_response
        })
    
    return pd.DataFrame(results)

# Run the test
test_results = test_response_generation(test_sample, model, tokenizer)

# Display the results
for idx, row in test_results.iterrows():
    print(f"Prompt: {row['prompt']}")
    print(f"Expected Response: {row['expected_response']}")
    print(f"Generated Response: {row['generated_response']}")
    print("-" * 50)

Prompt: I can't talk with  a human agent
Expected Response: Thank you for reaching out! I'm sorry to hear that you're facing difficulties in connecting with a human agent. Please rest assured that I'm here to assist you and resolve this issue. Could you please provide me with some more details about the problem you're experiencing? This will help me find the best solution for you.
Generated Response: I can't talk with  a human agent, but I can tell you that I'm not going to lie to you. I don't know what you're talking about."

"I'm sorry," she said. "I just want you to know that this is not the first time I've heard you say something like that. It's not like you've ever done anything like this before. You're just a normal human being, you know? I mean, I know it's hard to believe that you'd say that to someone like me. But I have to admit, it was hard for me when I first met you, because I didn't really know you at all, and I just wanted to make sure you were okay.
--------------------

## Integrate Knowledge Base

In [54]:
knowledge_base = {
    "refund_policy": "Our refund policy states that you can request a refund within 30 days of purchase...",
}

def generate_response_with_kb(prompt):
    if "refund policy" in prompt.lower():
        return knowledge_base["refund_policy"]
    else:
        return generate_response(prompt)

# Test the function with knowledge base integration
query = "What is your refund policy?"
response = generate_response_with_kb(query)
print(response)

Our refund policy states that you can request a refund within 30 days of purchase...


## Implement Escalation

In [56]:
def escalate_to_human(prompt):
    # Logic to escalate to human agent
    return "Your query has been escalated to a human agent. Please wait for a response."

def handle_query(prompt):
    if "complex issue" in prompt:  # Placeholder for more complex logic
        return escalate_to_human(prompt)
    else:
        return generate_response_with_kb(prompt)

# Test the complete system
query = "I have a complex issue with my account."
response = handle_query(query)
print(response)

Your query has been escalated to a human agent. Please wait for a response.


In [67]:
def generate_response(prompt, model, tokenizer, max_length=100):
    # Encode the prompt into tokens
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    
    # Check if the inputs tensor is empty
    if inputs.numel() == 0:
        return "Please provide a valid prompt."
    
    inputs = inputs.to(model.device)  # Move input to the same device as the model
    
    # Generate a response
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)
    
    # Decode the generated tokens to text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response

In [75]:
def generate_response(prompt, model, tokenizer, max_length=100):
    # Encode the prompt into tokens
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    
    # Check if the inputs tensor is empty
    if inputs.numel() == 0:
        return "Please provide a valid prompt."
    
    inputs = inputs.to(model.device)  # Move input to the same device as the model
    
    # Generate an attention mask
    attention_mask = torch.ones(inputs.shape, device=model.device)
    
    # Generate a response
    outputs = model.generate(inputs, attention_mask=attention_mask, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)
    
    # Decode the generated tokens to text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response

In [76]:
def chatbot_interface(model, tokenizer):
    print("Chatbot is ready! Type 'exit' to end the conversation.")
    while True:
        prompt = input("You: ")
        if prompt.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        response = generate_response(prompt, model, tokenizer)
        print(f"Chatbot: {response}")

# Run the chatbot interface
if __name__ == "__main__":
    chatbot_interface(model, tokenizer)

Chatbot is ready! Type 'exit' to end the conversation.
Chatbot: Goodbye!
